In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sentence_transformers import SentenceTransformer

/home/user03/Documents/SVN/UrFU.term3/Hackaton2023/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
faculty_fixed_df = pd.read_csv('faculty_fixed.csv', delimiter = ',', low_memory=False)

In [3]:
faculty_fixed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48865 entries, 0 to 48864
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   faculty_name        48865 non-null  object
 1   record_id           48865 non-null  int64 
 2   faculty_name_fixed  48865 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [4]:
faculty_fixed_translated_df = pd.read_csv('faculty_fixed_translated.csv', delimiter = ',', low_memory=False)

In [5]:
faculty_fixed_translated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44095 entries, 0 to 44094
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   faculty_name_fixed  44095 non-null  object 
 1   faculty_name_ru     44095 non-null  object 
 2   is_different        44095 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.0+ MB


In [6]:
faculty_fixed_translated_df

,faculty_name_fixed,faculty_name_ru,is_different
0,Юридический факультет,Юридический факультет,0.0
1,Экономический факультет,Экономический факультет,0.0
2,Факультет биоинженерии и биоинформатики,Факультет биоинженерии и биоинформатики,0.0
3,Биологический факультет,Биологический факультет,0.0
4,Высшая школа бизнеса,Высшая школа бизнеса,0.0
...,...,...,...
44090,Jay S. Sidhu School of Business and Leadership,Школа бизнеса и лидерства Джея С. Сидху,1.0
44091,Jaringan Telekomunikasi Digital,Цифровая телекоммуникационная сеть,1.0
44092,Japonistyka,японоведение,1.0
44093,James L. And Dorothy H. Dewar College of Educa...,Педагогический колледж Джеймса Л. и Дороти Х. ...,1.0


In [9]:
faculty_name_ru = list(faculty_fixed_translated_df['faculty_name_ru'].unique())

In [10]:
len(faculty_name_ru)

33459

In [11]:
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [12]:
embeddings = embedding_model.encode(faculty_name_ru)

In [13]:
embeddings.shape

(33459, 384)

In [14]:
embeddings

array([[ 0.11868316,  0.3599821 , -0.43437418, ..., -0.30043277,
         0.18584472,  0.26880696],
       [ 0.26872548,  0.1526852 , -0.23233958, ..., -0.37705183,
         0.13281727, -0.0325246 ],
       [ 0.06163373,  0.3901784 , -0.09501418, ..., -0.21069434,
         0.152418  , -0.11105683],
       ...,
       [-0.10577536,  0.10177018, -0.09970116, ..., -0.13156211,
        -0.27641532,  0.06510457],
       [ 0.04425332,  0.14655855, -0.06631096, ..., -0.13795738,
        -0.07695553, -0.12021367],
       [-0.21185961,  0.24273989, -0.05024791, ..., -0.22665648,
        -0.02291673,  0.30200568]], dtype=float32)

In [ ]:
'[' + (','.join(map(str, embeddings[0]))) + ']'

In [18]:
embeddings = [ '[' + (','.join(map(str, i))) + ']' for i in embeddings]

In [19]:
len(embeddings)

33459

In [20]:
embeddings[2]

'[0.06163373,0.3901784,-0.09501418,-0.04067063,-0.13706289,-0.23945917,-0.20263214,0.33040577,-0.06635491,0.19614625,0.1181526,-0.498805,-0.18659426,0.3301676,-0.2031801,-0.15205684,-0.15322122,0.050540764,0.19383922,-0.059436362,0.21142429,0.086635426,0.32208818,0.009856913,-0.102234535,0.009343837,-0.43532622,-0.08273537,-0.09880058,-0.23919652,0.23225503,0.045404054,0.22716601,-0.0328456,-0.041817643,0.28032482,-0.13310315,0.19729118,0.016667062,0.26813138,-0.079873994,-0.21171416,0.10194451,0.24329692,-0.067851275,-0.0115381675,-0.010190701,-0.12339389,-0.029465664,-0.520586,-0.11268935,-0.23588917,-0.11904854,-0.25583515,0.11891003,0.026501834,0.2386892,-0.15451045,-0.07170079,-0.014687309,0.3544845,0.0861568,-0.26977062,0.3857621,0.05622352,-0.18095121,0.20012407,0.23460439,0.29941237,-0.29744232,0.014155755,-0.30095583,0.17482989,0.14703867,0.24426055,0.37004936,0.03957125,0.04619773,0.41580436,-0.22481301,0.21232995,-0.06345312,0.13042682,0.111049406,0.03623992,0.3836533,-0.017

In [22]:
ru_vec_df = pd.DataFrame(columns = ['faculty_name_ru', 'embedding'])
for i in np.arange(len(faculty_name_ru)):
    ru_vec_df.loc[i] = [faculty_name_ru[i], embeddings[i]]

In [23]:
ru_vec_df

,faculty_name_ru,embedding
0,Юридический факультет,"[0.11868316,0.3599821,-0.43437418,-0.15627933,..."
1,Экономический факультет,"[0.26872548,0.1526852,-0.23233958,0.11820011,0..."
2,Факультет биоинженерии и биоинформатики,"[0.06163373,0.3901784,-0.09501418,-0.04067063,..."
3,Биологический факультет,"[-0.024020469,0.235017,-0.08868999,-0.02165936..."
4,Высшая школа бизнеса,"[0.35417813,0.26959077,-0.18569653,-0.01665383..."
...,...,...
33454,Школа бизнеса и лидерства Джея С. Сидху,"[0.06424836,0.33733055,-0.17458837,0.033753816..."
33455,Цифровая телекоммуникационная сеть,"[0.0063352357,-0.12739241,-0.012700283,-0.3998..."
33456,японоведение,"[-0.10577536,0.10177018,-0.09970116,0.03802707..."
33457,Педагогический колледж Джеймса Л. и Дороти Х. ...,"[0.04425332,0.14655855,-0.06631096,0.11734906,..."


In [29]:
joined_df = faculty_fixed_df.join(faculty_fixed_translated_df.set_index('faculty_name_fixed'), on='faculty_name_fixed', how='left')

In [30]:
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48865 entries, 0 to 48864
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   faculty_name        48865 non-null  object 
 1   record_id           48865 non-null  int64  
 2   faculty_name_fixed  48865 non-null  object 
 3   faculty_name_ru     48865 non-null  object 
 4   is_different        48865 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.9+ MB


In [35]:
joined_df.nunique()

faculty_name          48865
record_id              1026
faculty_name_fixed    44095
faculty_name_ru       33459
is_different              2
dtype: int64

In [26]:
joined_df

,faculty_name,record_id,faculty_name_fixed,faculty_name_ru,is_different
0,Юридический факультет,61006,Юридический факультет,Юридический факультет,0.0
1,Экономический факультет,59426,Экономический факультет,Экономический факультет,0.0
2,Факультет биоинженерии и биоинформатики,22719,Факультет биоинженерии и биоинформатики,Факультет биоинженерии и биоинформатики,0.0
3,Биологический факультет,21796,Биологический факультет,Биологический факультет,0.0
4,Высшая школа бизнеса,17360,Высшая школа бизнеса,Высшая школа бизнеса,0.0
...,...,...,...,...,...
48860,Jay S. Sidhu School of Business and Leadership,1,Jay S. Sidhu School of Business and Leadership,Школа бизнеса и лидерства Джея С. Сидху,1.0
48861,Jaringan Telekomunikasi Digital,1,Jaringan Telekomunikasi Digital,Цифровая телекоммуникационная сеть,1.0
48862,Japonistyka,1,Japonistyka,японоведение,1.0
48863,James L. And Dorothy H. Dewar College of Educa...,1,James L. And Dorothy H. Dewar College of Educa...,Педагогический колледж Джеймса Л. и Дороти Х. ...,1.0


In [37]:
ru_vec_df = ru_vec_df.join(joined_df.groupby('faculty_name_ru').agg(popularity=('record_id', 'sum')), on='faculty_name_ru', how='left')

In [38]:
ru_vec_df

,faculty_name_ru,embedding,popularity
0,Юридический факультет,"[0.11868316,0.3599821,-0.43437418,-0.15627933,...",63848
1,Экономический факультет,"[0.26872548,0.1526852,-0.23233958,0.11820011,0...",62745
2,Факультет биоинженерии и биоинформатики,"[0.06163373,0.3901784,-0.09501418,-0.04067063,...",22719
3,Биологический факультет,"[-0.024020469,0.235017,-0.08868999,-0.02165936...",21813
4,Высшая школа бизнеса,"[0.35417813,0.26959077,-0.18569653,-0.01665383...",17778
...,...,...,...
33454,Школа бизнеса и лидерства Джея С. Сидху,"[0.06424836,0.33733055,-0.17458837,0.033753816...",1
33455,Цифровая телекоммуникационная сеть,"[0.0063352357,-0.12739241,-0.012700283,-0.3998...",1
33456,японоведение,"[-0.10577536,0.10177018,-0.09970116,0.03802707...",1
33457,Педагогический колледж Джеймса Л. и Дороти Х. ...,"[0.04425332,0.14655855,-0.06631096,0.11734906,...",1


In [39]:
conn = psycopg2.connect(dbname="h_core", user="h_user", password="AUdZuMOGuP", host="84.201.178.7", port="55432")

In [32]:
with conn.cursor() as curs:
    for i in range(len(joined_df)):
        curs.execute('INSERT INTO h_faculty_2_ru (faculty_name, faculty_name_fixed_ru) VALUES (%s, %s)',
                     (joined_df.loc[i,'faculty_name'],
                      joined_df.loc[i,'faculty_name_ru']))
        if i % 1000 ==0:
            conn.commit()
            print(i)
    conn.commit()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000


In [41]:
with conn.cursor() as curs:
    for i in range(len(ru_vec_df)):
        curs.execute('INSERT INTO h_faculties_ru (faculty_name_fixed_ru, popularity, embedding) VALUES (%s, %s, %s)',
                     (ru_vec_df.loc[i,'faculty_name_ru'],
                      int(ru_vec_df.loc[i,'popularity']),
                      ru_vec_df.loc[i,'embedding']))
        if i % 1000 ==0:
            conn.commit()
            print(i)
    conn.commit()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000


In [42]:
conn.close()